In [1]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "anywidget>=0.9.0",
#     "jupyter-scatter-scsketch>=0.21.0",
#     "llvmlite>=0.44.0",
#     "numpy>=1.26.0",
#     "pandas>=2.0.0",
#     "scanpy>=1.9.0",
#     "scipy>=1.11.0",
#     "ipywidgets>=8.0.0",
#     "matplotlib>=3.7.0",
#     "requests>=2.31.0",
#     "watchfiles>=0.20.0",
# ]
#
# [tool.uv.sources]
# scsketch = { path = ".", editable = true }
# ///

# scSketch

> **Quick Start**: `uvx scsketch demo` (no cloning required!)  
> Or from this repo: `uvx juv run demo.ipynb`

scSketch provides a custom UI for [Jupyter-Scatter](https://jupyter-scatter.dev) that implements [Directional Analysis from Colubri et al's Sciviewer](https://doi.org/10.1093/bioinformatics/btab689). Sciviewer's directional analysis helps you interpret patterns in embedding visualizations by identifying genes varying locally along any user-specified direction.

For this demo, we're using the PBMC 3k (processed) dataset, since it is small (~2,600 cells), publicly available, and explicitly contains the UMAP coordinates and cluster metadata.

## Load Data

Load your single-cell data using scanpy and prepare it for visualization.

In [2]:
# Dataset download (shows a widget progress bar while downloading)

import os.path
import urllib.request

import ipywidgets as ipyw
from IPython.display import display

# Pick a dataset:
data_file = "1M_20260121.h5ad"
data_url = None  # set to a URL to download if the file is missing
# data_file = "pbmc3k.h5ad"
# data_url = "https://raw.githubusercontent.com/chanzuckerberg/cellxgene/main/example-dataset/pbmc3k.h5ad"

status = ipyw.HTML("")
bar = ipyw.IntProgress(value=0, min=0, max=100, description="Download", layout=ipyw.Layout(width="100%"))
ui = ipyw.VBox([status])
display(ui)

def _format_bytes(n: int) -> str:
    n = float(n)
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if n < 1024 or unit == "TB":
            return f"{int(n)} {unit}" if unit == "B" else f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

if os.path.exists(data_file):
    size = os.path.getsize(data_file)
    status.value = f"<b>Dataset:</b> Found <code>{data_file}</code> ({_format_bytes(size)})."
else:
    if not data_url:
        status.value = f"<b>Dataset:</b> Missing <code>{data_file}</code>. Set <code>data_url</code> to download."
    else:
        ui.children = [status, bar]
        status.value = f"<b>Dataset:</b> Downloading <code>{data_file}</code>…"

        def _hook(blocknum: int, blocksize: int, totalsize: int):
            downloaded = blocknum * blocksize
            if totalsize and totalsize > 0:
                pct = int(min(100, downloaded * 100 / totalsize))
                bar.value = pct
                status.value = (
                    f"<b>Dataset:</b> Downloading <code>{data_file}</code> — {pct}% "
                    f"({_format_bytes(min(downloaded, totalsize))} / {_format_bytes(totalsize)})"
                )
            else:
                status.value = f"<b>Dataset:</b> Downloading <code>{data_file}</code> — {_format_bytes(downloaded)}"

        urllib.request.urlretrieve(data_url, data_file, reporthook=_hook)
        bar.value = 100
        size = os.path.getsize(data_file)
        status.value = f"<b>Dataset:</b> Downloaded <code>{data_file}</code> ({_format_bytes(size)})."


In [3]:
import scanpy as sc

adata = sc.read(data_file)
adata

AnnData object with n_obs × n_vars = 1206761 × 32357
    obs: 'original_barcodes', 'cell_name', 'batch_id', 'pool_id', 'chip_id', 'well_id', 'n_genes', 'n_reads', 'n_umis', 'total_counts_mito', 'pct_counts_mito', 'doublet_score', 'predicted_AIFI_L1', 'AIFI_L1_score', 'AIFI_L1', 'predicted_AIFI_L2', 'AIFI_L2_score', 'AIFI_L2', 'predicted_AIFI_L3', 'AIFI_L3_score', 'AIFI_L3', 'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid', 'subject.cmv', 'subject.bmi', 'subject.race', 'subject.ethnicity', 'subject.birthYear', 'subject.ageAtFirstDraw', 'sample.visitName', 'sample.drawYear', 'sample.subjectAgeAtDraw', 'specimen.specimenGuid', 'pipeline.fileGuid', 'subject.ageGroup', 'vaccine_year', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'tissue_type', 'suspension_type', 'assay_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'cell_type_ontology_term_id', 'is_primary_data', 'cell_type'

## Launch scSketch

Create and display the scSketch widget.

In [4]:
from scsketch import ScSketch

sketch = ScSketch(
    adata=adata,
    metadata_cols=["louvain"],   # Metadata columns that can be used to color the cells
    color_by_default="louvain",  # A metadata column to use by default for coloring
    height=720,
    background_color="#111111",
)

sketch.show()

In [5]:
sketch.get_genes("Selection 1") # Get the list of selected genes as a dataframe for further analysis

""


## How to Use scSketch

1. **Select points**: Use the rectangle or lasso tool to select cells in the embedding
2. **Add selection**: Click the `+` button to save your selection
3. **Run analysis**: Click `Compute Directional Search` to identify genes varying along the selected direction
4. **Explore results**: Click on genes to see their Reactome pathways, and click pathways to view diagrams
5. **Color by genes**: Use the dropdown to color the embedding by specific genes or metadata
6. **Get list genes**: Use the get_genes() function in the scSketch object to retrieve the list of genes in a selected direction.

The directional analysis shows genes with their Pearson Correlation Coefficient (R) and p-value (p), representing which genes are most upregulated or downregulated along the selected direction.